## Морфологическая  дизамбигуация

Неоднозначность - одно из тех, свойств языка, которые делают его трудным (как для человеков так и для компьютеров.)

Неоднозначность проявляется на разных уровнях языка. И под каждую есть своя задача в NLP.  
Морфологическая неоднозначность - это когда одна и та же форма слова может иметь несколько вариантов морфологического описания.  
Например, ``стали`` - может быть глаголом в прошедшем времени мн.ч 3.л (``они стали``), а может - существительным женского рода в родительном падеже (``коробка из стали``).

Скорее всего, вы уже знаете или догадываетесь, что неоднозначность снимается в контексте.   
Однако контекст это не всегда несколько слов по соседству (как в примерах выше).   
Иногда это контекст находится в других, необязательно соседних предложениях.   
Например, предложение: ``Эти типы стали есть на складе.`` многозначно без другого предложения, в котором говорится о чём речь (о стали, или о типах).

Поэтому в теории - это очень сложная задача. И над ней работают многие комп. лингвисты.

Однако на практике эта задача либо вообще не стоит, либо решается достаточно хорошо.

Давайте посмотрим, почему:

Для русского есть готовые инструменты - pymorphy и mystem. И тот и другой умеют выдавать грамматическую информацию.

In [53]:
from lxml import etree
from pymorphy2 import MorphAnalyzer
from sklearn.metrics import classification_report
import numpy as np
from collections import Counter

Чтобы оценить как они справляются с неоднозначностью нам нужен размеченный корпус. А точнее корпус-снятник (т.е. тот в котором вручную разрешена неоднозначность). Обычно для этого используют НКРЯ, но там нужно запрашивать и подписывать какое-то соглашение. Поэтому мы возьмем OpenCorpora, который можно скачать без этих сложностей вот тут - http://opencorpora.org/?page=downloads (нужен снятник без UNK).

Сам корпус в xml. Для того, чтобы достать все в питоновские структуры данных, удобно использовать lxml и xpath.

In [10]:
open_corpora = etree.fromstring(open('data/annot.opcorpora.no_ambig_strict.xml', 'rb').read())

Так достанутся все предложения.

In [13]:
sentences = open_corpora.xpath('//tokens')

А так в отдельном предложении достанутся все слова.

In [21]:
tokens = sentences[0].xpath('token')

Для токена форма слова достается вот так:

In [22]:
tokens[0].xpath('@text')

['«']

А грамматическая информация вот так:

In [27]:
tokens[1].xpath('tfr/v/l/g/@v')

['NOUN', 'inan', 'femn', 'sing', 'nomn']

Соберем весь корпус в список. Для начала будем смотреть только на часть речи.

In [112]:
corpus = []


for sentence in open_corpora.xpath('//tokens'):
    sent_tagged = []
    for token in sentence.xpath('token'):
        word = token.xpath('@text')
        gram_info = token.xpath('tfr/v/l/g/@v')
        sent_tagged.append((word[0], gram_info[0]))
    
    corpus.append(sent_tagged)
        

In [113]:
len(corpus)

10551

In [114]:
corpus[0]

[('«', 'PNCT'),
 ('Школа', 'NOUN'),
 ('злословия', 'NOUN'),
 ('»', 'PNCT'),
 ('учит', 'VERB'),
 ('прикусить', 'INFN'),
 ('язык', 'NOUN')]

Воспользуемся pymorphy.

In [40]:
morph = MorphAnalyzer()

In [115]:
morph.parse('слово')[0].tag.POS

'NOUN'

Теперь просто пройдемся по каждому слову, предскажем его часть речи через пайморфи и сравним с тем, что стои в корпусе. Если совпадает добавим в список 1, если нет 0. Усреднив нули и единицы получим accuracy.

In [116]:
preds = []
mistakes = Counter()

for sent in corpus:
    for word, tag in sent:
        pred = str(morph.parse(word)[0].tag).split(',')[0].split(' ')[0]
        p = int(pred==tag)
        preds.append(p)
        if not p:
            mistakes.update([(word, tag, pred)])


Видно, что для части речи проблема неоднозначности особо и незаметна.

In [117]:
print(np.mean(preds))

0.9830159397798309


А если посмотреть на ошибки, то видно, что они происходят в каких-то не очень значимых случаях.

In [118]:
 mistakes.most_common(5)

[(('также', 'PRCL', 'CONJ'), 89),
 (('тоже', 'PRCL', 'ADVB'), 37),
 (('этом', 'ADJF', 'NPRO'), 36),
 (('Также', 'PRCL', 'CONJ'), 24),
 (('=', 'SYMB', 'UNKN'), 20)]

Попробуем теперь предсказывать сразу всю грамматическую информацию.

In [120]:
corpus = []


for sentence in open_corpora.xpath('//tokens'):
    sent_tagged = []
    for token in sentence.xpath('token'):
        word = token.xpath('@text')
        gram_info = token.xpath('tfr/v/l/g/@v')
        sent_tagged.append((word[0], set(gram_info)))
    
    corpus.append(sent_tagged)
        

In [121]:
preds = []
mistakes = Counter()

for sent in corpus:
    for word, tag in sent:
        pred = set(str(morph.parse(word)[0].tag).replace(' ', ',').split(','))
        p = len(pred&tag)/len(pred|tag)
        preds.append(p)
        if p < 0.5:
            mistakes.update([(word, tuple(tag), tuple(pred))])


Оценивание правда придется поменять. Так как тэгов несколько и они могут быть в разном порядке мы не можем просто их склеить. Поэтому посчитаем меру жаккара между множествами тэгов.

In [122]:
np.mean(preds)

0.9338109124901879

Она достаточно высокая.

А ошибки все те же.

In [125]:
mistakes.most_common(10)

[(('также', ('PRCL',), ('CONJ',)), 89),
 (('тоже', ('PRCL',), ('ADVB',)), 37),
 (('человек',
   ('NOUN', 'plur', 'masc', 'anim', 'gent'),
   ('masc', 'anim', 'nomn', 'NOUN', 'sing')),
  31),
 (('этом',
   ('Anph', 'masc', 'Apro', 'Subx', 'ADJF', 'loct', 'sing'),
   ('loct', 'NPRO', 'neut', 'sing')),
  27),
 (('Ссылки',
   ('plur', 'inan', 'femn', 'nomn', 'NOUN'),
   ('gent', 'femn', 'inan', 'NOUN', 'sing')),
  26),
 (('Также', ('PRCL',), ('CONJ',)), 24),
 (('Примечания',
   ('plur', 'inan', 'neut', 'nomn', 'NOUN'),
   ('NOUN', 'neut', 'inan', 'gent', 'sing')),
  23),
 (('=', ('SYMB',), ('UNKN',)), 20),
 (('№', ('SYMB',), ('UNKN',)), 19),
 (('>', ('SYMB',), ('UNKN',)), 19)]

Поэтому на практике, можно забить на неоднозначность.

Если все таки нужно (или хочется) разрешить неоднозначность - можно использовать mystem (там есть дизамбигуация). Но там своя токенизация и сложно будет оценивать качество на уже токенизированном корпусе.

Либо воспользоваться готовыми иструментами и обучить свой сниматель неоднозначности...